In [110]:
file = 'data/structral/airport/train.in'
N = 10

In [111]:
import tools

In [112]:
train_set, nodes, poss_set, neg_set = tools.sample_structural(file, N)

In [113]:
! cat data/bipartite/sample/train.in

1 2
1 4
3 4
5 2
7 2
7 6
7 0


# Get coordinates

In [114]:
from graph_tool.all import *
g = Graph(directed=False)
g.add_vertex(max(nodes) + 1)

<generator object <genexpr> at 0x7f42f5f51960>

In [115]:
a = 0
for edge in train_set:
    if edge not in poss_set:
        a += 1
        u, w = map(int, edge.split())
        g.add_edge(g.vertex(u), g.vertex(w))
print 1.0 * a / len(train_set)

0.900021249469


In [116]:
pos = sfdp_layout(g)

In [102]:
%matplotlib inline

graph_draw(g)

(<PropertyMap object with key type 'Vertex' and value type 'vector<double>', for Graph 0x7f42f6002e90, at 0x7f435064dcd0>,
 <PropertyMap object with key type 'Vertex' and value type 'bool', for Graph 0x7f42f6002e90, at 0x7f435064d450>)

In [117]:
from sklearn.metrics import roc_auc_score

In [118]:
features = tools.TopologicalFeatures(g, pos)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.dist])
roc_auc_score(Y, X)

0.9491245749799011

In [119]:
features = tools.TopologicalFeatures(g, pos)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.preferential_attachment])
roc_auc_score(Y, X)

0.94424561967513243

In [120]:
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random', random_state=0)

matrix = tools.make_sparse_matrix(train_set, nodes, poss_set)

features = tools.MFFeatures(model, matrix)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.score])
roc_auc_score(Y, X)

0.95075219005263811